In [1]:
!pip install gensim --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import gensim.downloader
import torch


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from dataset import EssayDataset, essay_collate_fn

In [5]:
BATCH_SIZE = 128
np.random.seed(5)

In [6]:
data = pd.read_csv("drive/MyDrive/data/cs7641/train_tokenized.csv") # Palash's file
data.drop(columns=["Unnamed: 0"], inplace=True)
data.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,word_token_nltk,sent_token,word_token_manual,clean_text,lemm_text,freq_dist,most_common_words,distinct_words_cnt
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,"['I', 'think', 'that', 'students', 'would', 'b...",['I think that students would benefit from lea...,"['I', 'think', 'that', 'students', 'would', 'b...","['think', 'students', 'would', 'benefit', 'lea...","['think', 'student', 'would', 'benefit', 'lear...",<FreqDist with 81 samples and 129 outcomes>,"[('student', 5), ('class', 5), ('go', 5)]",81
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,"['When', 'a', 'problem', 'is', 'a', 'change', ...",['When a problem is a change you have to let i...,"['When', 'a', 'problem', 'is', 'a', 'change', ...","['problem', 'change', 'let', 'best', 'matter',...","['problem', 'change', 'let', 'best', 'matter',...",<FreqDist with 80 samples and 215 outcomes>,"[('change', 16), ('different', 12), ('problem'...",80
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,"['Dear', ',', 'Principal', 'If', 'u', 'change'...","['Dear, Principal\n\nIf u change the school po...","['Dear,', 'Principal\n\nIf', 'u', 'change', 't...","['dear', 'principal', 'u', 'change', 'school',...","['dear', 'principal', 'u', 'change', 'school',...",<FreqDist with 58 samples and 133 outcomes>,"[('school', 9), ('average', 9), ('sport', 8)]",58
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,"['The', 'best', 'time', 'in', 'life', 'is', 'w...",['The best time in life is when you become you...,"['The', 'best', 'time', 'in', 'life', 'is', 'w...","['best', 'time', 'life', 'become', 'agree', 'g...","['best', 'time', 'life', 'become', 'agree', 'g...",<FreqDist with 132 samples and 282 outcomes>,"[('make', 16), ('choice', 10), ('others', 8)]",132
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,"['Small', 'act', 'of', 'kindness', 'can', 'imp...",['Small act of kindness can impact in other pe...,"['Small', 'act', 'of', 'kindness', 'can', 'imp...","['small', 'act', 'kindness', 'impact', 'people...","['small', 'act', 'kindness', 'impact', 'people...",<FreqDist with 67 samples and 112 outcomes>,"[('people', 6), ('person', 6), ('act', 5)]",67


In [7]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-200')
glove_dict = glove_vectors.key_to_index
len(glove_dict)

[==================================================] 100.0% 252.1/252.1MB downloaded


400000

In [8]:
gensim_emb_weights = glove_vectors.vectors

In [9]:
data['list_lower_word_token_nltk'] = data['word_token_nltk'].apply(eval)
data['list_lower_word_token_nltk'] = data['list_lower_word_token_nltk'].apply(lambda l: [e.lower() for e in l if e not in ('', ' ')])
data['num_valid_token'] = data['list_lower_word_token_nltk'].apply(lambda x: len([c for c in x if c in glove_dict]))
data['num_all_token'] = data['list_lower_word_token_nltk'].apply(len)
data['unrecgonized_tokens'] = data['list_lower_word_token_nltk'].apply(lambda x: [c for c in x if c not in glove_dict])
data["word_token_nltk_idx"] = data['list_lower_word_token_nltk'].apply(
    lambda x: [glove_vectors.key_to_index[c]  if c in glove_dict else 400000 for c in x])

In [10]:
data["word_token_nltk_idx"]

0       [41, 269, 12, 543, 54, 2161, 25, 2741, 22, 163...
1       [61, 7, 747, 14, 7, 511, 81, 33, 4, 886, 20, 8...
2       [8607, 1, 2965, 83, 6479, 511, 0, 164, 527, 3,...
3       [0, 254, 79, 6, 214, 14, 61, 81, 399, 4961, 2,...
4       [357, 743, 3, 20524, 86, 1433, 6, 68, 69, 86, ...
                              ...                        
3906    [41, 733, 622, 28319, 6, 905, 10, 631, 95, 91,...
3907    [500, 1485, 1, 543, 88, 36, 33, 4, 4117, 17, 6...
3908    [28, 7, 747, 14, 7, 1019, 10, 81, 4, 88, 392, ...
3909    [109, 69, 10027, 17, 4324, 28833, 9, 8643, 1, ...
3910    [88, 81, 269, 12, 1899, 14, 0, 444, 873, 10, 6...
Name: word_token_nltk_idx, Length: 3911, dtype: object

In [11]:
task_cols = ["cohesion","syntax","vocabulary", 
    "phraseology", "grammar", "conventions"]
data.loc[3, task_cols].tolist()

[4.5, 4.5, 4.5, 4.5, 4.0, 5.0]

In [12]:
np.random.seed(42)
df_train, df_valid, df_test = np.split(
    data[["text_id", "full_text", "word_token_nltk_idx"] + task_cols].sample(
        frac=1, random_state=42), 
    [int(.8*len(data)), int(.9*len(data))])

In [13]:
df_train.head()

,text_id,full_text,word_token_nltk_idx,cohesion,syntax,vocabulary,phraseology,grammar,conventions
1552,772D27D400BB,It god to have a possitive attitude when you d...,"[20, 1533, 4, 33, 7, 400000, 4191, 61, 81, 88,...",3.0,2.5,2.5,2.0,2.0,2.0
2114,9E8F3C6405CA,Why do people ask more then one person for adv...,"[738, 88, 69, 1712, 56, 127, 48, 899, 10, 3240...",3.0,2.0,3.0,3.5,3.0,3.0
1965,948771F795EB,"We accomplish more when we are active, and are...","[53, 9749, 56, 61, 53, 32, 1546, 1, 5, 32, 690...",4.0,4.0,3.0,4.0,4.0,4.0
3856,FE14D7378CFB,Do you agree or disagree about imagination bei...,"[88, 81, 2137, 46, 10027, 59, 9201, 134, 56, 4...",3.0,3.0,3.5,3.0,3.5,3.5
1610,7AAE019F70D6,I disagree with the principal saying that all ...,"[41, 10027, 17, 0, 2965, 345, 12, 64, 1813, 18...",3.5,3.5,3.5,3.5,3.0,3.5


In [14]:
ds_train = EssayDataset(df_train, "word_token_nltk_idx", task_cols)
dl_train = torch.utils.data.DataLoader(
    ds_train, batch_size=BATCH_SIZE, 
    shuffle=True, collate_fn=essay_collate_fn)
ds_valid = EssayDataset(df_valid, "word_token_nltk_idx", task_cols)
dl_valid = torch.utils.data.DataLoader(
    ds_valid, batch_size=BATCH_SIZE, 
    shuffle=True, collate_fn=essay_collate_fn)
ds_test = EssayDataset(df_test, "word_token_nltk_idx", task_cols)
dl_test = torch.utils.data.DataLoader(
    ds_test, batch_size=BATCH_SIZE, 
    shuffle=True, collate_fn=essay_collate_fn)

In [15]:
df_train.to_parquet("drive/MyDrive/data/cs7641/df_train.pq")
df_valid.to_parquet("drive/MyDrive/data/cs7641/df_valid.pq")
df_test.to_parquet("drive/MyDrive/data/cs7641/df_test.pq")

In [16]:
from gru import GRUGrader
from trainer import train_grader

In [17]:
gru_sizes = [32, 64, 128]
gru_depth = [4, 6, 8]
bidirectional = [True, False]
decoder_depth = [2,3,4]
decoder_size = {
    2: [256, 64],
    3: [256, 512, 64],
    4: [256, 512, 128, 64]
}

model_versions = {
    # Default
    "gru_64_6__bidirectiona_True__decoder_3":{
        "gru_size":64,
        "gru_num_layer":6,
        "bidirectional":True,
        "decoder_depth":3,
        "decoder_size":[256, 512, 64]
    },
    # Experiment with gru size
    "gru_32_6__bidirectiona_True__decoder_3":{
        "gru_size":32,
        "gru_num_layer":6,
        "bidirectional":True,
        "decoder_depth":3,
        "decoder_size":[256, 512, 64]
    },
    "gru_128_6__bidirectiona_True__decoder_3":{
        "gru_size":128,
        "gru_num_layer":6,
        "bidirectional":True,
        "decoder_depth":3,
        "decoder_size":[256, 512, 64]
    },
    # Experiment with gru depth
    "gru_64_4__bidirectiona_True__decoder_3":{
        "gru_size":64,
        "gru_num_layer":4,
        "bidirectional":True,
        "decoder_depth":3,
        "decoder_size":[256, 512, 64]
    },
    "gru_64_8__bidirectiona_True__decoder_3":{
        "gru_size":64,
        "gru_num_layer":8,
        "bidirectional":True,
        "decoder_depth":3,
        "decoder_size":[256, 512, 64]
    },
    # Experiment with bidirectional = False
    "gru_64_6__bidirectiona_False__decoder_3":{
        "gru_size":64,
        "gru_num_layer":6,
        "bidirectional":False,
        "decoder_depth":3,
        "decoder_size":[256, 512, 64]
    },
    # Experiment with decoder depth
    "gru_64_6__bidirectiona_True__decoder_2":{
        "gru_size":64,
        "gru_num_layer":6,
        "bidirectional":True,
        "decoder_depth":2,
        "decoder_size":[256,64]
    },
    "gru_64_6__bidirectiona_True__decoder_4":{
        "gru_size":64,
        "gru_num_layer":6,
        "bidirectional":True,
        "decoder_depth":4,
        "decoder_size":[256, 512, 128, 64]
    },
}

In [18]:
for name, params in model_versions.items():
    print(name)
    gru_grader = GRUGrader(gensim_emb_weights=gensim_emb_weights, **params)
    best_model = train_grader(dl_train, dl_valid, 
        model=gru_grader,
        model_name=name,
        save_path="drive/MyDrive/data/cs7641/rnn_models",
        loss_func=torch.nn.MSELoss(),
        opt=torch.optim.Adam(gru_grader.parameters(), lr=0.001),
        epochs=30)
    print("--------------")

gru_64_6__bidirectiona_True__decoder_3
Epoch 0000, train loss: 0.592711, valid loss: 0.584031, best valid loss: inf
Epoch 0001, train loss: 0.430979, valid loss: 0.384714, best valid loss: inf
Epoch 0002, train loss: 0.418822, valid loss: 0.406971, best valid loss: inf
Epoch 0003, train loss: 0.410147, valid loss: 0.373027, best valid loss: inf
Epoch 0004, train loss: 0.396715, valid loss: 0.352475, best valid loss: inf
Epoch 0005, train loss: 0.376862, valid loss: 0.336481, best valid loss: inf
Epoch 0006, train loss: 0.334355, valid loss: 0.324591, best valid loss: inf
Epoch 0007, train loss: 0.338426, valid loss: 0.343952, best valid loss: 0.343952
Epoch 0008, train loss: 0.305288, valid loss: 0.298866, best valid loss: 0.343952
Epoch 0009, train loss: 0.289566, valid loss: 0.315413, best valid loss: 0.315413
Epoch 0010, train loss: 0.277197, valid loss: 0.314843, best valid loss: 0.314843
Epoch 0011, train loss: 0.277565, valid loss: 0.322955, best valid loss: 0.314843
Epoch 0012, 

In [19]:

model_versions2 = {
    # # Experiment with gru depth
    "gru_64_12__bidirectiona_True__decoder_3":{
        "gru_size":64,
        "gru_num_layer":12,
        "bidirectional":True,
        "decoder_depth":3,
        "decoder_size":[256, 512, 64]
    },
    "gru_64_16__bidirectiona_True__decoder_3":{
        "gru_size":64,
        "gru_num_layer":16,
        "bidirectional":True,
        "decoder_depth":3,
        "decoder_size":[256, 512, 64]
    },
    # Experiment with decoder size
    "gru_64_12__bidirectiona_True__decoder_3_lite":{
        "gru_size":64,
        "gru_num_layer":12,
        "bidirectional":True,
        "decoder_depth":3,
        "decoder_size":[128, 256, 64]
    },
    "gru_64_16__bidirectiona_True__decoder_3_lite":{
        "gru_size":64,
        "gru_num_layer":16,
        "bidirectional":True,
        "decoder_depth":3,
        "decoder_size":[128, 256, 64]
    },
    # Experiment with decoder depth
    "gru_64_12__bidirectiona_True__decoder_4_lite":{
        "gru_size":64,
        "gru_num_layer":12,
        "bidirectional":True,
        "decoder_depth":4,
        "decoder_size":[128, 256, 128, 64]
    },
    "gru_64_16__bidirectiona_True__decoder_4_lite":{
        "gru_size":64,
        "gru_num_layer":16,
        "bidirectional":True,
        "decoder_depth":4,
        "decoder_size":[128, 256, 128, 64]
    },
}

In [20]:
for name, params in model_versions2.items():
    print(name)
    gru_grader = GRUGrader(gensim_emb_weights=gensim_emb_weights, **params)
    best_model = train_grader(dl_train, dl_valid, 
        model=gru_grader,
        model_name=name,
        save_path="drive/MyDrive/data/cs7641/rnn_models",
        loss_func=torch.nn.MSELoss(),
        opt=torch.optim.Adam(gru_grader.parameters(), lr=0.001),
        epochs=30)
    print("--------------")

gru_64_12__bidirectiona_True__decoder_3
Epoch 0000, train loss: 0.654184, valid loss: 0.613599, best valid loss: inf
Epoch 0001, train loss: 0.432877, valid loss: 0.389186, best valid loss: inf
Epoch 0002, train loss: 0.421245, valid loss: 0.367764, best valid loss: inf
Epoch 0003, train loss: 0.416117, valid loss: 0.356951, best valid loss: inf
Epoch 0004, train loss: 0.406878, valid loss: 0.413490, best valid loss: 0.413490
Epoch 0005, train loss: 0.394382, valid loss: 0.337150, best valid loss: 0.413490
Epoch 0006, train loss: 0.368025, valid loss: 0.343445, best valid loss: 0.413490
Epoch 0007, train loss: 0.333652, valid loss: 0.386843, best valid loss: 0.386843
Epoch 0008, train loss: 0.328986, valid loss: 0.328809, best valid loss: 0.386843
Epoch 0009, train loss: 0.292586, valid loss: 0.278185, best valid loss: 0.386843
Epoch 0010, train loss: 0.293106, valid loss: 0.398448, best valid loss: 0.386843
Epoch 0011, train loss: 0.280748, valid loss: 0.325965, best valid loss: 0.325